# ducklake-pandas Tutorial

This notebook walks through all the main features of `ducklake-pandas` — a pure-Python Pandas integration for DuckLake catalogs.

We'll:
1. Create a DuckLake catalog with DuckDB and populate it
2. Read it with ducklake-pandas (eager reads, time travel, schema evolution)
3. Write data back with ducklake-pandas (INSERT, DELETE, UPDATE, MERGE)
4. Verify everything from DuckDB
5. Explore DDL, partitioning, data inlining, views, and catalog maintenance

In [ ]:
# Install/upgrade to latest versions
!pip install -U ducklake-pandas duckdb

In [ ]:
import os
import shutil
import tempfile

import duckdb
import pandas as pd

# Work in a temp directory so we don't pollute the repo
WORKDIR = tempfile.mkdtemp(prefix="ducklake_tutorial_")
CATALOG = os.path.join(WORKDIR, "catalog.ducklake")
DATA = os.path.join(WORKDIR, "data")
os.makedirs(DATA, exist_ok=True)

print(f"Working directory: {WORKDIR}")
print(f"Catalog: {CATALOG}")
print(f"Data path: {DATA}")

## 1. Initialize the Catalog with DuckDB

DuckLake catalogs must be initialized by DuckDB's DuckLake extension. This creates the metadata schema (snapshot tables, column tables, etc.) that ducklake-pandas reads and writes.

In [ ]:
con = duckdb.connect()
con.execute("INSTALL ducklake; LOAD ducklake")
con.execute(f"ATTACH 'ducklake:sqlite:{CATALOG}' AS lake (DATA_PATH '{DATA}/')")

# Create a table and insert some data
con.execute("""
    CREATE TABLE lake.users (
        id INTEGER,
        name VARCHAR,
        email VARCHAR,
        score DOUBLE,
        active BOOLEAN
    )
""")

con.execute("""
    INSERT INTO lake.users VALUES
        (1, 'Alice',   'alice@example.com',   95.5, true),
        (2, 'Bob',     'bob@example.com',     87.3, true),
        (3, 'Carol',   'carol@example.com',   72.1, false),
        (4, 'Dave',    'dave@example.com',     91.0, true),
        (5, 'Eve',     'eve@example.com',      68.5, false)
""")

print("DuckDB: Created table and inserted 5 users")
con.execute("SELECT * FROM lake.users ORDER BY id").fetchdf()

In [ ]:
# Also create an events table for later
con.execute("""
    CREATE TABLE lake.events (
        event_id INTEGER,
        user_id INTEGER,
        region VARCHAR,
        amount DOUBLE,
        ts TIMESTAMP
    )
""")

con.execute("""
    INSERT INTO lake.events VALUES
        (1, 1, 'us',   100.0, '2025-01-15 10:00:00'),
        (2, 2, 'eu',   250.0, '2025-01-15 11:00:00'),
        (3, 1, 'us',   75.0,  '2025-01-16 09:00:00'),
        (4, 3, 'apac', 300.0, '2025-01-16 14:00:00'),
        (5, 4, 'eu',   150.0, '2025-01-17 08:00:00')
""")

# Close DuckDB so ducklake-pandas can access the catalog
con.close()
print("DuckDB: Created events table with 5 rows. Connection closed.")

## 2. Reading with ducklake-pandas

No DuckDB needed — reads metadata from SQLite directly and scans Parquet files through Pandas.

In [ ]:
from ducklake_pandas import read_ducklake

# Read the full table
df = read_ducklake(CATALOG, "users")
print("read_ducklake — all users:")
df

In [ ]:
# Filter and sort with standard pandas operations
result = df[(df["active"] == True) & (df["score"] > 90)][["name", "score"]]
result = result.sort_values("score", ascending=False)
print("Active users with score > 90:")
result

In [ ]:
# Select specific columns via projection
df = read_ducklake(CATALOG, "users", columns=["id", "name"])
print("Projection — only id and name:")
df

## 3. Catalog Inspection

The `DuckLakeCatalog` class provides Python equivalents of DuckLake's utility functions.

In [ ]:
from ducklake_pandas import DuckLakeCatalog

catalog = DuckLakeCatalog(CATALOG)

print("=== Snapshots ===")
print(catalog.snapshots())

print("\n=== Current Snapshot ===")
print(catalog.current_snapshot())

print("\n=== Schemas ===")
print(catalog.list_schemas())

print("\n=== Tables ===")
print(catalog.list_tables())

In [ ]:
print("=== Table Info ===")
print(catalog.table_info())

print("\n=== Files in 'users' ===")
print(catalog.list_files("users"))

print("\n=== Catalog Settings ===")
print(catalog.settings())

## 4. Writing with ducklake-pandas

### 4a. Append rows

> **Note:** DuckDB created `id` as `INTEGER` (Int32). When appending with Pandas, use matching
> dtypes (e.g. `pd.array([6, 7], dtype="int32")`) to avoid schema mismatches.

In [ ]:
from ducklake_pandas import write_ducklake

new_users = pd.DataFrame({
    "id": pd.array([6, 7], dtype="int32"),
    "name": ["Frank", "Grace"],
    "email": ["frank@example.com", "grace@example.com"],
    "score": [88.0, 94.5],
    "active": [True, True],
})

write_ducklake(new_users, CATALOG, "users", mode="append",
               author="tutorial", commit_message="Add Frank and Grace")

print("After append — 7 users:")
read_ducklake(CATALOG, "users").sort_values("id")

### 4b. Verify from DuckDB

DuckDB can read what ducklake-pandas wrote — full interop.

In [ ]:
con = duckdb.connect()
con.execute("LOAD ducklake")
con.execute(f"ATTACH 'ducklake:sqlite:{CATALOG}' AS lake (DATA_PATH '{DATA}/')")

print("DuckDB reads ducklake-pandas data:")
con.execute("SELECT * FROM lake.users ORDER BY id").fetchdf()

In [ ]:
con.close()  # release for ducklake-pandas

### 4c. Delete rows

In [ ]:
from ducklake_pandas import delete_ducklake

deleted = delete_ducklake(CATALOG, "users",
                          lambda df: df["active"] == False,
                          author="tutorial", commit_message="Remove inactive users")
print(f"Deleted {deleted} inactive users")

print("\nRemaining users:")
read_ducklake(CATALOG, "users").sort_values("id")

### 4d. Update rows

In [ ]:
from ducklake_pandas import update_ducklake

updated = update_ducklake(
    CATALOG, "users",
    updates={"score": 100.0, "email": "alice-updated@example.com"},
    predicate=lambda df: df["name"] == "Alice",
    author="tutorial", commit_message="Perfect score for Alice"
)
print(f"Updated {updated} rows")

print("\nAlice's new data:")
df = read_ducklake(CATALOG, "users")
df[df["name"] == "Alice"]

### 4e. Merge (upsert)

In [ ]:
from ducklake_pandas import merge_ducklake

source = pd.DataFrame({
    "id": pd.array([1, 2, 8], dtype="int32"),
    "name": ["Alice", "Bob", "Heidi"],
    "email": ["alice@new.com", "bob@new.com", "heidi@example.com"],
    "score": [98.0, 92.0, 89.0],
    "active": [True, True, True],
})

rows_updated, rows_inserted = merge_ducklake(
    CATALOG, "users", source, on="id",
    when_matched_update=True,
    when_not_matched_insert=True,
    author="tutorial", commit_message="Merge user updates"
)
print(f"Updated: {rows_updated}, Inserted: {rows_inserted}")

print("\nAll users after merge:")
read_ducklake(CATALOG, "users").sort_values("id")

### 4f. Verify from DuckDB again

In [ ]:
con = duckdb.connect()
con.execute("LOAD ducklake")
con.execute(f"ATTACH 'ducklake:sqlite:{CATALOG}' AS lake (DATA_PATH '{DATA}/')")

print("DuckDB sees all ducklake-pandas changes:")
con.execute("SELECT * FROM lake.users ORDER BY id").fetchdf()

In [ ]:
con.close()

## 5. Time Travel

Every write creates a new snapshot. We can read the table at any historical point.

In [ ]:
# See all snapshots
catalog = DuckLakeCatalog(CATALOG)
snapshots = catalog.snapshots()
print("All snapshots:")
print(snapshots)

# Find the first snapshot that has user data
for v in sorted(snapshots["snapshot_id"].tolist()):
    try:
        df = read_ducklake(CATALOG, "users", snapshot_version=v)
        if len(df) > 0:
            print(f"\nSnapshot {v} — original data ({len(df)} rows):")
            print(df.sort_values("id"))
            break
    except Exception:
        continue

print(f"\nLatest snapshot — current data:")
print(read_ducklake(CATALOG, "users").sort_values("id"))

## 6. Change Data Feed

Track what changed between snapshots.

In [ ]:
snap_ids = sorted(catalog.snapshots()["snapshot_id"].tolist())
start_v = snap_ids[0]
end_v = snap_ids[-1]

print(f"Changes between snapshot {start_v} and {end_v}:")
changes = catalog.table_changes("users", start_version=start_v, end_version=end_v)
changes

## 7. Schema Evolution (DDL)

### Add, rename, and drop columns

In [ ]:
from ducklake_pandas import (
    alter_ducklake_add_column,
    alter_ducklake_rename_column,
    alter_ducklake_drop_column,
)

# Add a column — existing rows get NULL
alter_ducklake_add_column(CATALOG, "users", "department", "varchar",
                          author="tutorial", commit_message="Add department column")

print("After ADD COLUMN 'department':")
print(read_ducklake(CATALOG, "users").sort_values("id"))

In [ ]:
# Rename a column
alter_ducklake_rename_column(CATALOG, "users", "email", "contact_email",
                              author="tutorial", commit_message="Rename email to contact_email")

print("After RENAME COLUMN 'email' to 'contact_email':")
print(read_ducklake(CATALOG, "users").sort_values("id"))

In [ ]:
# Drop a column
alter_ducklake_drop_column(CATALOG, "users", "department",
                            author="tutorial", commit_message="Drop department column")

print("After DROP COLUMN 'department':")
print(read_ducklake(CATALOG, "users").sort_values("id"))

In [ ]:
# DuckDB sees the schema changes
con = duckdb.connect()
con.execute("LOAD ducklake")
con.execute(f"ATTACH 'ducklake:sqlite:{CATALOG}' AS lake (DATA_PATH '{DATA}/')")

print("DuckDB DESCRIBE after schema evolution:")
con.execute("DESCRIBE lake.users").fetchdf()

In [ ]:
con.close()

## 8. Partitioned Writes

Set partitioning on a table, then inserts automatically create Hive-style directory layouts.

In [ ]:
from ducklake_pandas import alter_ducklake_set_partitioned_by

# Partition the events table by region
alter_ducklake_set_partitioned_by(CATALOG, "events", ["region"],
                                   author="tutorial", commit_message="Partition events by region")

# Insert more events — they'll be written as partitioned Parquet files
# Note: DuckDB created event_id/user_id as INTEGER (Int32)
new_events = pd.DataFrame({
    "event_id": pd.array([6, 7, 8, 9], dtype="int32"),
    "user_id": pd.array([1, 2, 6, 7], dtype="int32"),
    "region": ["us", "eu", "us", "apac"],
    "amount": [200.0, 175.0, 50.0, 400.0],
    "ts": pd.to_datetime(["2025-02-01 10:00:00", "2025-02-01 11:00:00",
                          "2025-02-02 09:00:00", "2025-02-02 14:00:00"]),
})

write_ducklake(new_events, CATALOG, "events", mode="append",
               author="tutorial", commit_message="Partitioned event insert")

print("All events:")
read_ducklake(CATALOG, "events").sort_values("event_id")

In [ ]:
# Filter by region — partition pruning applies when reading
all_events = read_ducklake(CATALOG, "events")
us_events = all_events[all_events["region"] == "us"]
print("US events only:")
us_events.sort_values("event_id")

In [ ]:
# Show the partitioned file structure
print("Data files on disk:")
for root, dirs, files in os.walk(DATA):
    for f in files:
        path = os.path.join(root, f)
        rel = os.path.relpath(path, DATA)
        size = os.path.getsize(path)
        print(f"  {rel}  ({size:,} bytes)")

## 9. Data Inlining

Small inserts can be stored directly in the catalog database instead of creating tiny Parquet files.

In [ ]:
from ducklake_pandas import create_ducklake_table

# Create a table for small writes
create_ducklake_table(CATALOG, "metrics", {
    "ts": "timestamp",
    "sensor": "varchar",
    "value": "double",
})

# Small inserts get inlined (stored in SQLite, not Parquet)
for i in range(5):
    row = pd.DataFrame({
        "ts": pd.to_datetime([f"2025-03-01 {10+i}:00:00"]),
        "sensor": [f"sensor_{i}"],
        "value": [20.0 + i * 1.5],
    })
    write_ducklake(row, CATALOG, "metrics", mode="append",
                   data_inlining_row_limit=10)

print("Metrics (5 inlined rows, no Parquet files):")
print(read_ducklake(CATALOG, "metrics").sort_values("ts"))

# Check: no Parquet files for metrics yet
catalog = DuckLakeCatalog(CATALOG)
print("\nFiles for 'metrics':")
print(catalog.list_files("metrics"))

In [ ]:
# A larger insert triggers flush to Parquet
big_batch = pd.DataFrame({
    "ts": pd.to_datetime([f"2025-03-02 {h:02d}:00:00" for h in range(24)]),
    "sensor": [f"sensor_{i % 5}" for i in range(24)],
    "value": [15.0 + i * 0.5 for i in range(24)],
})

write_ducklake(big_batch, CATALOG, "metrics", mode="append",
               data_inlining_row_limit=10)

print(f"Metrics after big insert: {len(read_ducklake(CATALOG, 'metrics'))} rows")
print("\nFiles for 'metrics' (now has Parquet):")
catalog = DuckLakeCatalog(CATALOG)
print(catalog.list_files("metrics"))

## 10. CREATE TABLE AS

Create a new table with data in a single atomic snapshot.

In [ ]:
from ducklake_pandas import create_table_as_ducklake

# Build a summary from existing data
events = read_ducklake(CATALOG, "events")
summary = (
    events.groupby("region")
    .agg(total_amount=("amount", "sum"), event_count=("event_id", "count"))
    .reset_index()
    .sort_values("region")
)

create_table_as_ducklake(summary, CATALOG, "region_summary",
                          author="tutorial", commit_message="Regional summary CTAS")

print("Created 'region_summary' via CTAS:")
read_ducklake(CATALOG, "region_summary")

## 11. Views

In [ ]:
from ducklake_pandas import create_ducklake_view, drop_ducklake_view

# Create a view
create_ducklake_view(
    CATALOG, "active_users",
    "SELECT id, name, score FROM users WHERE active = true",
    author="tutorial", commit_message="Create active_users view"
)
print("Created view 'active_users'")

# DuckDB can query it
con = duckdb.connect()
con.execute("LOAD ducklake")
con.execute(f"ATTACH 'ducklake:sqlite:{CATALOG}' AS lake (DATA_PATH '{DATA}/')")

print("\nDuckDB querying the view:")
con.execute("SELECT * FROM lake.active_users ORDER BY id").fetchdf()

In [ ]:
# Replace view
create_ducklake_view(
    CATALOG, "active_users",
    "SELECT id, name, score FROM users WHERE active = true AND score > 90",
    or_replace=True,
    author="tutorial", commit_message="Update view: score > 90 filter"
)

# Need to re-attach for DuckDB to see the updated view
con.close()
con = duckdb.connect()
con.execute("LOAD ducklake")
con.execute(f"ATTACH 'ducklake:sqlite:{CATALOG}' AS lake (DATA_PATH '{DATA}/')")

print("Updated view (score > 90 only):")
con.execute("SELECT * FROM lake.active_users ORDER BY id").fetchdf()

In [ ]:
con.close()

## 12. Schema and Table Management

In [ ]:
from ducklake_pandas import (
    create_ducklake_schema,
    drop_ducklake_schema,
    rename_ducklake_table,
    drop_ducklake_table,
)

# Create a new schema
create_ducklake_schema(CATALOG, "staging")

# Create a table in it
create_ducklake_table(CATALOG, "raw_data", {
    "id": "bigint",
    "payload": "varchar",
}, schema="staging")

print("Schemas:", DuckLakeCatalog(CATALOG).list_schemas()["schema_name"].tolist())
print("Tables in 'staging':", DuckLakeCatalog(CATALOG).list_tables(schema="staging")["table_name"].tolist())

# Rename the table
rename_ducklake_table(CATALOG, "raw_data", "raw_events", schema="staging")
print("After rename:", DuckLakeCatalog(CATALOG).list_tables(schema="staging")["table_name"].tolist())

# Drop schema with cascade
drop_ducklake_schema(CATALOG, "staging", cascade=True)
print("After drop cascade:", DuckLakeCatalog(CATALOG).list_schemas()["schema_name"].tolist())

## 13. Catalog Maintenance

Clean up old snapshots and orphaned files.

In [ ]:
from ducklake_pandas import expire_snapshots, vacuum_ducklake

catalog = DuckLakeCatalog(CATALOG)
print(f"Snapshots before cleanup: {len(catalog.snapshots())}")

# Keep only the last 3 snapshots
expired = expire_snapshots(CATALOG, keep_last_n=3)
print(f"Expired {expired} snapshots")

catalog = DuckLakeCatalog(CATALOG)
print(f"Snapshots after cleanup: {len(catalog.snapshots())}")

# Delete orphaned Parquet files
deleted = vacuum_ducklake(CATALOG)
print(f"Deleted {deleted} orphaned Parquet files")

## 14. Final Verification: Full Roundtrip

Let's verify the complete state from both DuckDB and ducklake-pandas.

In [ ]:
# ducklake-pandas view
print("=== ducklake-pandas ===")
for table in ["users", "events", "metrics", "region_summary"]:
    df = read_ducklake(CATALOG, table)
    print(f"\n{table}: {len(df)} rows")
    print(df.head(3))

In [ ]:
# DuckDB view
con = duckdb.connect()
con.execute("LOAD ducklake")
con.execute(f"ATTACH 'ducklake:sqlite:{CATALOG}' AS lake (DATA_PATH '{DATA}/')")

print("=== DuckDB ===")
for table in ["users", "events", "metrics", "region_summary"]:
    result = con.execute(f"SELECT COUNT(*) FROM lake.{table}").fetchone()
    print(f"{table}: {result[0]} rows")

print("\nDuckDB — users:")
con.execute("SELECT * FROM lake.users ORDER BY id").fetchdf()

In [ ]:
con.close()

## Cleanup

In [ ]:
# Remove the temp directory
shutil.rmtree(WORKDIR)
print(f"Cleaned up {WORKDIR}")

---

## Summary

This tutorial covered:

| Feature | Functions Used |
|---------|---------------|
| **Reading** | `read_ducklake` |
| **Writing** | `write_ducklake` (error/append/overwrite) |
| **Deletes** | `delete_ducklake` |
| **Updates** | `update_ducklake` |
| **Merge** | `merge_ducklake` |
| **CTAS** | `create_table_as_ducklake` |
| **DDL** | `create_ducklake_table`, `alter_ducklake_add/drop/rename_column` |
| **Partitioning** | `alter_ducklake_set_partitioned_by` |
| **Data inlining** | `write_ducklake(..., data_inlining_row_limit=N)` |
| **Views** | `create_ducklake_view`, `drop_ducklake_view` |
| **Schemas** | `create_ducklake_schema`, `drop_ducklake_schema` |
| **Time travel** | `read_ducklake(..., snapshot_version=N)` |
| **Change data feed** | `DuckLakeCatalog.table_changes()` |
| **Catalog inspection** | `DuckLakeCatalog.snapshots/table_info/list_files/...` |
| **Maintenance** | `expire_snapshots`, `vacuum_ducklake` |
| **DuckDB interop** | Full bidirectional |

For the complete API reference, see the [wiki](https://github.com/pdet/ducklake-pandas/wiki).